<a href="https://colab.research.google.com/github/ulfboge/gee-landcover-masking/blob/main/combine_rasters_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Combine Rasters in Google Colab
This notebook allows you to combine raster files by area with each year's data placed in separate bands. Follow the instructions in each cell.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install GDAL in Google Colab
%%capture
!apt-get install -y gdal-bin python3-gdal python3-rasterio
!pip install gdal

In [ ]:
import os
from osgeo import gdal
from google.colab import files

# Create directories
data_dir = "/content/drive/MyDrive/earthengine"
output_folder = "/content/drive/MyDrive/NDFI_Combined" # Changed this line to create the folder inside 'MyDrive'
os.makedirs(data_dir, exist_ok=True)
os.makedirs(output_folder, exist_ok=True)

In [ ]:
# Upload filelist.txt
print("Upload filelist.txt")
uploaded = files.upload()
filelist_path = list(uploaded.keys())[0]

# Read filelist.txt
with open(filelist_path, 'r') as file:
    raster_files = [line.strip() for line in file.readlines()]

In [ ]:
# Find raster files in data_dir
raster_files_found = [f for f in os.listdir(data_dir) if f.endswith('.tif')]

# Print found raster files
print("Found the following raster files in", data_dir + ":")
for file in raster_files_found:
    print(file)

In [ ]:
# Group files by area
area_files = {}
for raster in raster_files:
    parts = raster.split("_")
    year = parts[2]  # Extract year
    area_id = parts[-1].replace(".tif", "")  # Extract area ID
    if area_id not in area_files:
        area_files[area_id] = []
    area_files[area_id].append((year, raster))

In [ ]:
# Process each area
for area_id, files in area_files.items():
    # Sort files by year
    files.sort(key=lambda x: x[0])

    # Get full file paths
    file_paths = [os.path.join(data_dir, f[1]) for f in files]

    # Open raster datasets
    datasets = [gdal.Open(fp) for fp in file_paths]

    # Check if any dataset is None
    if any(ds is None for ds in datasets):
        print(f"Error: Could not open one or more raster files for area {area_id}.")
        print("File paths:")
        for fp in file_paths:
            print(fp)
        continue  # Skip to the next area

    # Get raster dimensions and projection
    cols = datasets[0].RasterXSize
    rows = datasets[0].RasterYSize
    projection = datasets[0].GetProjection()
    geotransform = datasets[0].GetGeoTransform()

    # Create output file
    output_file = os.path.join(output_folder, f"Combined_Area_{area_id}.tif")
    driver = gdal.GetDriverByName('GTiff')
    out_ds = driver.Create(output_file, cols, rows, len(datasets), gdal.GDT_Float32)
    out_ds.SetProjection(projection)
    out_ds.SetGeoTransform(geotransform)

    # Write each year's data to a separate band
    for i, ds in enumerate(datasets):
        out_band = out_ds.GetRasterBand(i + 1)
        data = ds.GetRasterBand(1).ReadAsArray()
        out_band.WriteArray(data)
        out_band.SetNoDataValue(-9999)

    # Close datasets
    out_ds = None
    for ds in datasets:
        ds = None

print(f"Rasters combined successfully and saved to {output_folder}.")

In [ ]:
# Zip and download the combined files
!zip -r /content/NDFI_Combined.zip /content/NDFI_Combined
files.download('/content/NDFI_Combined.zip')